# Quantum Integrated Information (QII)

Seguiremos el paper "Integrated Information-induced quantum collapse". Primero, necesitaremos una función que pase de estados tipo $|01\rangle$ a vectores en la base computacional.

In [1]:
"""
El input es un vector en la notación ket y el output un vector en la base computacional.
"""
function quantum_to_vec(quantum_state::Array{Int64,1})
    n=2^length(quantum_state)
    vec=zeros(Int64,n)
    pos=parse(Int64,join(quantum_state),2)+1
    vec[pos]=1
    vec
end

quantum_to_vec

Verifiquemos lo anterior obteniendo la matriz de densidad de un estado puro, en este caso el Greenberger–Horne–Zeilinger (GHZ).
Recordemos que para un estado puro $\rho=|\psi\rangle \langle \psi |$

In [2]:
psi_GHZ=(quantum_to_vec([0,0,0])+quantum_to_vec([1,1,1]))/sqrt(2);
ρGHZ=psi_GHZ*psi_GHZ'

8×8 Array{Float64,2}:
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5

## Traza parcial
Hagamos ahora una función de la traza parcial generalizada para cualquier dimensión de sistemas sea cual sea la partición.Para esto traduzco el código 
http://library.wolfram.com/infocenter/MathSource/8763/#downloads
de mathematica language a julia.
Primero, necesito hacer una función que, dado un arreglo y dos posiciones de este, intercambie las partes.

Notemos que la función dTraceSystemSymb se usa cuando la matriz de densidad es un arreglo de strings, mientras que dTraceSystem es para cuando el la matriz de densidad es un arreglo numérico. La notación es como sigue:

Si $\rho_{ABC}$ es el sistema general, entonces, por ejemplo, $\rho_{B}=Tr_{AC}\rho_{ABC}=dTraceSystem(\rho_{ABC},[1,3],dim)$, donde dim=2 si se trata de qubits, dim=3 si son qutrits, dim=d si son qudits.


In [9]:
function SwapParts(expr,pos1::Int64,pos2::Int64)
    expr[pos1], expr[pos2]=expr[pos2], expr[pos1]
    expr
end

function IntegerDigits(n,b,len)
    map(parse,split(base(b,n,len),""))
end

function FromDigits(list,base)
    parse(Int64,join(list),base)
end

function ++(a::Array{String,1},b::Array{String,1})
    c=Array{String}(length(a))
    for i in 1:length(a)
        char1=a[i]
        char2=b[i]
        c[i]="$char1 + $char2"
    end
    c
end
    

++ (generic function with 1 method)

In [10]:
function dTraceSystemSymb(D::Array{String,2}, s, dim)
    Qudits=reverse(sort(s));
    TrkM=D;
    z=length(Qudits)+1;
    q=1;
    while q<z
        n=Int(log(dim,size(TrkM)[1]));
        M=deepcopy(TrkM);
        k=Qudits[q]
        if k==n
            TrkM=[];
            p=1;
            C=[]
            while p<(dim^n)+1
                C=M[p,:][1:dim:dim^n]
                for h in 1:dim-1
                    C=C++M[p+h,:][1+h:dim:dim^n]
                    if p==1
                        append!(TrkM,C)
                    else
                        TrkM=hcat(TrkM,C)
                    end
                end
                p+=dim
            end
            TrkM=permutedims(TrkM,[2,1])
        else
            j=0
            while j<n-k
                b=[0];
                i=1
                while i<(dim^n)+1
                    if IntegerDigits(i-1,dim,n)[n]!=IntegerDigits(i-1,dim,n)[n-j-1] && count(k->(k==i),b)==0
                        b=append!(b,FromDigits(SwapParts((IntegerDigits(i-1,dim,n)),n,n-j-1),dim)+1)
                        c=collect(1:1:dim^n)
                        perm=SwapParts(c,i,FromDigits(SwapParts(IntegerDigits(i-1,dim,n),n,n-j-1),dim)+1)
                        M=M[perm,perm]
                    end
                    i+=1
                end
                j+=1
            end
            TrkM=[];
            p=1
            while p<(dim^n)+1
                C=M[p,:][1:dim:dim^n]
                for h in 1:dim-1
                    C=C++M[p+h,:][1+h:dim:dim^n]
                    if p==1
                        append!(TrkM,C)
                    else
                        TrkM=hcat(TrkM,C)
                    end
                end
                p+=dim
            end
            TrkM=permutedims(TrkM,[2,1])
        end
        q+=1
    end
    return TrkM
end

function dTraceSystem(D, s, dim)
    Qudits=reverse(sort(s));
    TrkM=D;
    z=length(Qudits)+1;
    q=1;
    while q<z
        n=Int(log(dim,size(TrkM)[1]));
        M=deepcopy(TrkM);
        k=Qudits[q]
        if k==n
            TrkM=[];
            p=1;
            C=[]
            while p<(dim^n)+1
                C=M[p,:][1:dim:dim^n]
                for h in 1:dim-1
                    C=C+M[p+h,:][1+h:dim:dim^n]
                    if p==1
                        append!(TrkM,C)
                    else
                        TrkM=hcat(TrkM,C)
                    end
                end
                p+=dim
            end
            TrkM=permutedims(TrkM,[2,1])
        else
            j=0
            while j<n-k
                b=[0];
                i=1
                while i<(dim^n)+1
                    if IntegerDigits(i-1,dim,n)[n]!=IntegerDigits(i-1,dim,n)[n-j-1] && count(k->(k==i),b)==0
                        b=append!(b,FromDigits(SwapParts((IntegerDigits(i-1,dim,n)),n,n-j-1),dim)+1)
                        c=collect(1:1:dim^n)
                        perm=SwapParts(c,i,FromDigits(SwapParts(IntegerDigits(i-1,dim,n),n,n-j-1),dim)+1)
                        M=M[perm,perm]
                    end
                    i+=1
                end
                j+=1
            end
            TrkM=[];
            p=1
            while p<(dim^n)+1
                C=M[p,:][1:dim:dim^n]
                for h in 1:dim-1
                    C=C+M[p+h,:][1+h:dim:dim^n]
                    if p==1
                        append!(TrkM,C)
                    else
                        TrkM=hcat(TrkM,C)
                    end
                end
                p+=dim
            end
            TrkM=permutedims(TrkM,[2,1])
        end
        q+=1
    end
    return convert(Array{Float64,2},TrkM)
end

dTraceSystem (generic function with 1 method)

Trato ahora de comprobar esta función de distintas maneras.|

In [6]:
ρa=["a11" "a12"; "a21" "a22"]
ρb=["b11" "b12"; "b21" "b22"]
ρc=["c11" "c12"; "c21" "c22"]
ρabc=kron(kron(ρa,ρb),ρc)
dTraceSystemSymb(ρabc,[1,3],2)

LoadError: LoadError: MethodError: no method matching ++(::Array{Any,1}, ::Array{Any,1})
while loading In[6], in expression starting on line 5

Veamos si los resultados obtenidos corresponden a los presentados en Integrated Information-induced quantum collapse. Exploremos primero el estado GHZ.

In [11]:
ρGHZ_A=dTraceSystem(ρGHZ,[2,3],2)
ρGHZ_B=dTraceSystem(ρGHZ,[1,3],2)
ρGHZ_C=dTraceSystem(ρGHZ,[1,2],2)
kron(kron(ρGHZ_A,ρGHZ_B),ρGHZ_C)

8×8 Array{Float64,2}:
 0.125  0.0    0.0    0.0    0.0    0.0    0.0    0.0  
 0.0    0.125  0.0    0.0    0.0    0.0    0.0    0.0  
 0.0    0.0    0.125  0.0    0.0    0.0    0.0    0.0  
 0.0    0.0    0.0    0.125  0.0    0.0    0.0    0.0  
 0.0    0.0    0.0    0.0    0.125  0.0    0.0    0.0  
 0.0    0.0    0.0    0.0    0.0    0.125  0.0    0.0  
 0.0    0.0    0.0    0.0    0.0    0.0    0.125  0.0  
 0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.125

In [12]:
ρGHZ_A=dTraceSystem(ρGHZ,[2,3],2)
ρGHZ_BC=dTraceSystem(ρGHZ,[1],2)
A=kron(ρGHZ_A,ρGHZ_BC)

8×8 Array{Float64,2}:
 0.25  0.0  0.0  0.0   0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.25  0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.25  0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.0   0.0  0.0  0.0 
 0.0   0.0  0.0  0.0   0.0   0.0  0.0  0.25

In [17]:
ρGHZ

8×8 Array{Float64,2}:
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5

In [16]:
logm(A)

8×8 Array{Complex{Float64},2}:
 NaN+0.0im  NaN+0.0im  NaN+0.0im  …  NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im  …  NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im
 NaN+0.0im  NaN+0.0im  NaN+0.0im     NaN+0.0im  NaN+0.0im  NaN+0.0im

Veamos ahora qué pasa con el estado W.

In [13]:
psi_W=(quantum_to_vec([0,0,1])+quantum_to_vec([0,1,0])+quantum_to_vec([1,0,0]))/sqrt(3);
ρW=psi_W*psi_W';
ρW_A=dTraceSystem(ρW,[2,3],2)
ρW_B=dTraceSystem(ρW,[1,3],2)
ρW_C=dTraceSystem(ρW,[1,2],2)
27*kron(kron(ρW_A,ρW_B),ρW_C)

8×8 Array{Float64,2}:
 8.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [14]:
ρW_A=dTraceSystem(ρW,[2,3],2)
ρW_BC=dTraceSystem(ρW,[1],2)
9*kron(ρW_A,ρW_BC)

8×8 Array{Float64,2}:
 2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  2.0  2.0  0.0  0.0  0.0  0.0  0.0
 0.0  2.0  2.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

Vemos que todo concuerda con el paper.

## Combinatoria y cálculo de MIP
En este apartado se genran todas las particiones posibles de un sistema

In [121]:
using Combinatorics

INFO: Recompiling stale cache file /home/humberto/.julia/lib/v0.5/Polynomials.ji for module Polynomials.


In [142]:
collect(partitions([1,2,3]))[2:end]

4-element Array{Any,1}:
 Array{Int64,1}[[1,2],[3]]  
 Array{Int64,1}[[1,3],[2]]  
 Array{Int64,1}[[1],[2,3]]  
 Array{Int64,1}[[1],[2],[3]]

En el caso cuántico, el MIP está definido como el ínfimo de la entropía cuántica relativa entre la matriz de densidad del sistema completo y el producto de Kronecker de alguna partición posible del sistema, es decir,
$$\Phi(\rho)=inf(S(\rho|| \otimes^N _{i=1} Tr_{i} (\rho )): {H}=H_1 \otimes ... \otimes H_N )$$
y la entroía cuántica relativa está dada por 
$$S(\sigma_1 || \sigma_2 ) := Tr(\sigma_1 \log(\sigma_1 ))-Tr(\sigma_1 \log(\sigma_2 ))$$
donde $\sigma_1$ y $\sigma_2$ son matrices de densidad. Esta definición es una extensión directa de la entropía de Kullback-Leibler.

Para calcular el logaritmo de una matriz, utilicemos el algoritmo de la wikipedia https://en.wikipedia.org/wiki/Logarithm_of_a_matrix: $\log A = V(\log A') V^{-1}$

In [289]:
function chopnumber(x::Float64, epsilon=10e-15)   
    if abs(x) < epsilon
        return Float64(0)
    else
        return x
    end
end

function chopnumber(y::Array{Float64,2}, epsilon=10e-15) 
    x=copy(y)
    for i=1:length(x)
        x[i]=chopnumber(x[i],epsilon)
    end
    return x
end

function notinf(x::Float64,sust=10e15)   
    if abs(x) == Inf
        return sign(x)*sust
    else
        return x
    end
end

function notinf(y::Array{Float64,2},sust=10e15) 
    x=copy(y)
    for i=1:length(x)
        x[i]=notinf(x[i],sust)
    end
    return x
end

function NaNtozero(y::Array{Complex{Float64},2}) 
    x=copy(y)
    for i=1:length(x)
        if isequal(NaN,abs(x[i]))
            x[i]=0
        end
    end
    return x
end

NaNtozero (generic function with 2 methods)

In [208]:
function quantum_relative_entropy(A,B)
    trace(A*Mlog(A))-trace(A*Mlog(B))
end

quantum_relative_entropy (generic function with 1 method)

In [290]:
NaNtozero(logm(A))

8×8 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im